In [14]:
import os
import cv2
import numpy as np
import pickle
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
empty_path = '/kaggle/input/parking-lot-detection-counter/parking/clf-data/empty'
not_empty_path = '/kaggle/input/parking-lot-detection-counter/parking/clf-data/not_empty'
img_size = (15, 15)

In [16]:
def load_images(folder, label):
    features = []
    labels = []
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        img = cv2.imread(path)
        if img is None:
            continue
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normalize to [0, 1]
        
        features.append(img.flatten())
        labels.append(label)
    
    return features, labels

# Apply to both classes (no augmentation)
X_empty, y_empty = load_images(empty_path, 0)
X_not_empty, y_not_empty = load_images(not_empty_path, 1)

# Combine data
X = np.array(X_empty + X_not_empty)
y = np.array(y_empty + y_not_empty)

print("Data shape:", X.shape)


Data shape: (6090, 675)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.svm import SVC

model = SVC(kernel='rbf', class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       651
           1       1.00      1.00      1.00       567

    accuracy                           1.00      1218
   macro avg       1.00      1.00      1.00      1218
weighted avg       1.00      1.00      1.00      1218



In [18]:
print("Total samples:")
print("Empty:", len(X_empty))
print("Not Empty:", len(X_not_empty))

Total samples:
Empty: 3045
Not Empty: 3045


In [19]:
import joblib

# Save the model to a .joblib file
joblib.dump(model, "svm_parking_model_rgb_15x15_new.joblib")

print("Model saved as svm_parking_model_rgb_15x15.joblib")


Model saved as svm_parking_model_rgb_15x15.joblib
